In [2]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    # !pip install -q -U tfx==0.15.0rc0
    print("You can safely ignore the package incompatibility errors.")
except Exception:
    pass
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
print(tf.__version__)
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import datasets
from datetime import datetime 
from tensorflow.keras import datasets


tf.random.set_seed(1228)
np.random.seed(1228)

TensorFlow 2.x selected.
You can safely ignore the package incompatibility errors.
2.1.0


In [0]:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [88]:
print(train_dataset.element_spec)
print(test_dataset.element_spec)


(TensorSpec(shape=(28, 28), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))
(TensorSpec(shape=(28, 28), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))


In [0]:
train_dataset = train_dataset.shuffle(1024)

In [0]:
@tf.function
def preprocess(x, y):
  '''
  convert to float
  expand dim <- ignore batch for the loader
  scaling 
  '''
  x = tf.cast(x, 'float')
  x = tf.expand_dims(x, 2)
  x = (x - 127.5) / 127.5
  return x, y

In [0]:
## Checker 

# counter = 0 
# for data, label in dataset:
#   print(data)
#   print('=====')
#   print(label)
#   counter += 1
#   if counter == 1:
#     break

In [0]:
train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

In [0]:
## Checker 

# counter = 0 
# for data, label in dataset:
#   print(data)
#   print('=====')
#   print(label)
#   counter += 1
#   if counter == 1:
#     break

In [0]:
train_dataset = train_dataset.batch(512).prefetch(10)
test_dataset = test_dataset.batch(512).prefetch(10)

In [0]:
## Checker 

# counter = 0 
# for data, label in dataset:
#   print(data)
#   print('=====')
#   print(label)
#   counter += 1
#   if counter == 1:
#     break

In [94]:
# base model

input_ = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(128, 7, 1, padding='same', activation='relu')(input_)
# x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.Conv2D(128, 3, 1, padding='same', activation='relu')(x)
# x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.Conv2D(128, 3, 1, padding='same', activation='relu')(x)
# x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.Conv2D(128, 3, 1, padding='same', activation='relu')(x)
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.models.Model(input_, x)
model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])


logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
my_call = tf.keras.callbacks.TensorBoard(log_dir=logdir)

hist = model.fit(train_dataset, epochs=20, validation_data=test_dataset, callbacks=[my_call])

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 28, 28, 128)       6400      
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 28, 28, 128)       147584    
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 28, 28, 128)       147584    
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 28, 28, 128)       147584    
_________________________________________________________________
global_average_pooling2d_8 ( (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256

## Class Layer

In [0]:
class MyDense(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super(MyDense self).__init__(**kwargs)
    pass
  def build(self, input_shape):
    pass
  def call(self, input):
    pass
  def compute_output_shape(selmf input_shape):
    pass
  def get_config(self):
    base_config = super().get_config()
    return {**base_config}

In [0]:
class MyMultilayer(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super(MyMultilayer, self).__init__(**kwargs)
  
  def call(self, x):
    x1, x2 = x
    return [x1+x2, x1*x2, x1/x2]
  
  def compute_output_shape(self, input_shape):
    b1, b2 = input_shape
    return [b1, b1, b1]

In [0]:
class MyGaussianNoise(tf.keras.layers.Layer):
  def __init__(self, stddev, **kwargs):
    super(MyGaussianNoise, self).__init__(**kwargs)
    self.stddev = stddev
  
  def call(self, x, training=None):
    if training:
      noise = tf.random.normal(tf.shape(x), stddev=self.stddev)
      return x + noise
    else :
      return x

  def compute_output_shape(self, input_shape):
    return input_shape


In [0]:
class ResidualBlock(tf.keras.layers.Layer):
  def __init__(self, n_layers, n_neurons, **kwargs):
    super(ResidualBlock, self).__init__(**kwargs)
    self.hidden = [tf.keras.layers.Dense(n_neurons, activation='elu', kernel_initializer='he_normal') for _ in range(n_layers)]

  def call(self, input):
    z = inputs
    for layer in self.hidden:
      z = layer(z)
    return inputs + z
  

In [0]:
class ResidualRegressor(tf.keras.Model):
  def __init__(self, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.h1 = tf.keras.layers.Dense(30, activation='elu', kernel_initializer='he_normal')
    self.r1 = ResidualBlock(2, 30)
    self.r2 = ResidualBlock(2, 30)
    self.out = tf.keras.layers.Dense(output_dim)

  def call(self, x):
    z = self.h1(x)
    for _ in range(1 + 3):
      z = self.r1(z)
    z = self.r2(z)
    return self.out(z)
    